# Running a optimum path finder as navigation tool across ice of certain thickness

In [1]:
from polar_bearings.configuration import Location
from polar_bearings.data_ingest import import_nc_to_pandas, single_data_df, clean_ice_thickness_df
from polar_bearings.heatmap import plot_heatmap
from polar_bearings.opt_pah_finder_robotics.potential_field_planning import potential_field_planning
from polar_bearings.opt_pah_finder_robotics.navigate_ice import process_data

In [2]:
start_location = Location(
    name = 'Ulukhaktok',
    longitude=70.746225,
    latitude=-117.821145
)

In [3]:
destination_location = Location(
    name = "Sachs Harbor",
    longitude = 71.985805,
    latitude = -125.370926
)

In [4]:
data = import_nc_to_pandas(file='cmems_mod_arc_phy_anfc_nextsim_hm_1636224873602.nc')

In [5]:
single_day = single_data_df(data)

In [6]:
single_day = clean_ice_thickness_df(single_day)

In [27]:
single_day.sort_values('longitude').head()

,longitude,latitude,sithick
15581,-127.976639,70.715698,0.670070
15582,-127.966927,70.741699,0.491899
15583,-127.957184,70.767693,0.491899
15584,-127.947426,70.793694,0.608666
15585,-127.937630,70.819695,0.479838


In [8]:
plotmap = plot_heatmap(single_day,
        axis_min_lat = single_day.latitude.mean(),
        axis_max_lon = single_day.longitude.mean(),
        start_location = start_location,
        destination_location = destination_location)

In [9]:
display(plotmap)

#### Running the optimal path finding algorithm

In [31]:

rescaling_factor: int = 1

gx, gy, sx, sy, ox, oy = process_data(single_day, rescaling_factor, safety_threshold=2)

grid_size = 0.1  # potential grid size [m]
robot_radius = 0.1  # robot radius [m]

# path generation
rx, ry = potential_field_planning(sx, sy, gx, gy, ox, oy, grid_size, robot_radius)



Calculating potential field: 100%|██████████| 402/402 [04:31<00:00,  1.48it/s]

Goal!!


In [33]:
import numpy as np
# we need to rescale the coordinates back to longitude and latitude
# This still needs some investigation

ycoords = np.array(ry) + start_location.latitude
xccords = np.array(rx) + start_location.longitude

In [34]:
import pandas as pd
coodrs = pd.DataFrame(ycoords, xccords).reset_index()
coodrs.values.tolist()

[[80.93219912109375, -117.16516965820313],
 [80.846225, -117.021145],
 [80.746225, -116.921145],
 [80.646225, -116.821145],
 [80.54622499999999, -116.721145],
 [80.446225, -116.621145],
 [80.346225, -116.521145],
 [80.246225, -116.421145],
 [80.146225, -116.321145],
 [80.04622499999999, -116.221145],
 [79.946225, -116.121145],
 [79.846225, -116.021145],
 [79.746225, -115.921145],
 [79.646225, -115.921145],
 [79.54622499999999, -115.921145],
 [79.446225, -115.921145],
 [79.346225, -115.921145],
 [79.246225, -115.921145],
 [79.146225, -115.921145],
 [79.04622499999999, -115.921145],
 [78.946225, -115.921145],
 [78.846225, -115.921145],
 [78.746225, -115.921145],
 [78.646225, -115.921145],
 [78.54622499999999, -115.921145],
 [78.446225, -115.921145],
 [78.346225, -115.921145],
 [78.246225, -115.921145],
 [78.146225, -115.921145],
 [78.04622499999999, -115.921145],
 [77.946225, -115.921145],
 [77.846225, -115.921145],
 [77.746225, -115.921145],
 [77.646225, -115.921145],
 [77.5462249999999

In [35]:
#import folium
#
#f1=folium.FeatureGroup("Route")
#line_1=folium.vector_layers.PolyLine(
#    coodrs,
#    popup='<b>Optimal Route</b>',
#    tooltip='Route',
#    color='black',
#    weight=10
#).add_to(f1)
#f1.add_to(plotmap)
#display(plotmap)